In [1]:
# set run environment (local/colab)
# if colab install required packages and set appropriate root_path
import os
from pathlib import Path
if os.getenv("COLAB_RELEASE_TAG"):
    colab = True
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = Path('/content/drive/Othercomputers/My computer/EQILLM/')
    !pip install -r '/content/drive/Othercomputers/My computer/EQILLM/requirements.txt'
    !pip install transformers[torch]
    !pip install accelerate -U
    !pip install datasets
    !pip install torchinfo
    import sys
    sys.path.append(root_path)
    #ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U
else:
    colab = False
    root_path = Path('')


import itertools
import pandas as pd
import openai
import datetime
import os
import csv
import matplotlib.pyplot as plt

from dotenv import load_dotenv, dotenv_values
from tqdm.notebook import tqdm_notebook
from eqillm import finetune, get_log_for_val, validate, val_metrics, yeelight_eow_notification, param_combinations, load_PolarIs, df_to_ds


dotenv_config = dotenv_values(root_path / '.env')
yeelight_notify = dotenv_config['YEELIGHT_NOTIFY'] if 'YEELIGHT_NOTIFY' in dotenv_config else False

## Load and filter

In [7]:
training_logs = pd.read_csv(os.path.join(root_path, 'output/training_logs.csv'))

# shorten all runs to max 30 epochs
training_logs = training_logs[training_logs['Epoch']<=30]

# drop all runs which have less than 3 epochs
training_logs = training_logs.groupby(['model', 'timestamp']).filter(lambda x: len(x)>=3)
training_logs

In [11]:
load_PolarIs('data\PolarIs-Pathos.xlsx')